# 10. Generate Realistic Slow Queries and Metrics

In [1]:
import pandas as pd

# Paths
users_csv = r"C:\Users\pc\data science\SADOP\data\Users.csv"
accounts_csv = r"C:\Users\pc\data science\SADOP\data\Accounts.csv"
transactions_csv = r"C:\Users\pc\data science\SADOP\data\Transactions.csv"
logs_csv = r"C:\Users\pc\data science\SADOP\data\Logs.csv"

# Load data
df_users = pd.read_csv(users_csv)
df_accounts = pd.read_csv(accounts_csv)
df_transactions = pd.read_csv(transactions_csv)
df_logs = pd.read_csv(logs_csv)

print("Tables loaded:")
print("Users:", len(df_users))
print("Accounts:", len(df_accounts))
print("Transactions:", len(df_transactions))
print("Logs:", len(df_logs))


Tables loaded:
Users: 10000
Accounts: 19968
Transactions: 250188
Logs: 79849


## 10.1 Simulate Large Joins and Aggregates


In [2]:
import time
import random

# Already loaded CSVs
# df_users, df_accounts, df_transactions

metrics = []

# Simulate 1000 heavy queries (adjust depending on performance)
for _ in range(1000):
    start_time = time.time()
    
    # Randomly choose a user
    user_id = random.choice(df_users['user_id'].values)
    
    # Large join: Users -> Accounts -> Transactions
    joined = df_users[df_users['user_id'] == user_id] \
             .merge(df_accounts, on='user_id', how='left') \
             .merge(df_transactions, on='account_id', how='left')
    
    # Aggregate example: sum of transactions per account
    agg = joined.groupby('account_id')['amount'].sum()
    
    end_time = time.time()
    exec_time = end_time - start_time
    rows_examined = len(joined)
    
    metrics.append({
        "query": f"SUM transactions for user_id={user_id}",
        "query_time": exec_time,
        "rows_examined": rows_examined,
        "joins": 2  # Users->Accounts->Transactions
    })

df_metrics = pd.DataFrame(metrics)
df_metrics.head()


,query,query_time,rows_examined,joins
0,SUM transactions for user_id=1615,0.055942,23,2
1,SUM transactions for user_id=7965,0.026148,21,2
2,SUM transactions for user_id=5778,0.023165,24,2
3,SUM transactions for user_id=7861,0.029534,20,2
4,SUM transactions for user_id=7450,0.023048,13,2


In [3]:
df_metrics.sort_values(by = "query_time",ascending=False)

,query,query_time,rows_examined,joins
83,SUM transactions for user_id=5133,0.063969,11,2
0,SUM transactions for user_id=1615,0.055942,23,2
84,SUM transactions for user_id=7806,0.049126,57,2
138,SUM transactions for user_id=8979,0.037707,47,2
135,SUM transactions for user_id=2753,0.037560,29,2
...,...,...,...,...
286,SUM transactions for user_id=8986,0.018635,11,2
292,SUM transactions for user_id=3108,0.018530,19,2
293,SUM transactions for user_id=6719,0.018524,32,2
264,SUM transactions for user_id=2174,0.018214,28,2


## 10.2 Simulate Missing Index Effect


In [4]:
df_transactions_shuffled = df_transactions.sample(frac=1).reset_index(drop=True)

metrics_missing_index = []

for _ in range(2000):
    start_time = time.time()
    
    # Join shuffled Transactions to simulate slow scan
    joined = df_accounts.merge(df_transactions_shuffled, on='account_id', how='left')
    
    agg = joined.groupby('user_id')['amount'].sum()
    
    end_time = time.time()
    exec_time = end_time - start_time
    rows_examined = len(joined)
    
    metrics_missing_index.append({
        "query": "SUM transactions (simulated missing index)",
        "query_time": exec_time,
        "rows_examined": rows_examined,
        "joins": 1
    })

df_metrics_missing = pd.DataFrame(metrics_missing_index)
df_metrics_missing.head()


,query,query_time,rows_examined,joins
0,SUM transactions (simulated missing index),0.095660,250188,1
1,SUM transactions (simulated missing index),0.077518,250188,1
2,SUM transactions (simulated missing index),0.070966,250188,1
3,SUM transactions (simulated missing index),0.071256,250188,1
4,SUM transactions (simulated missing index),0.071192,250188,1


In [5]:
df_metrics_missing.shape


(2000, 4)

## 10.3 Simulate Traffic Spikes


In [6]:
combined_metrics = pd.concat([df_metrics, df_metrics_missing], ignore_index=True)

# Simulate spike by running each query multiple times
spike_metrics = []

for idx, row in combined_metrics.iterrows():
    for _ in range(3):  # 3x load
        exec_time = row['query_time'] * random.uniform(1, 1.5)  # slight variability
        spike_metrics.append({
            "query": row['query'],
            "query_time": exec_time,
            "rows_examined": row['rows_examined'],
            "joins": row['joins']
        })

df_spike_metrics = pd.DataFrame(spike_metrics)
df_spike_metrics


,query,query_time,rows_examined,joins
0,SUM transactions for user_id=1615,0.062965,23,2
1,SUM transactions for user_id=1615,0.070396,23,2
2,SUM transactions for user_id=1615,0.081162,23,2
3,SUM transactions for user_id=7965,0.034609,21,2
4,SUM transactions for user_id=7965,0.032294,21,2


In [10]:
df_spike_metrics.sort_values(by = "query_time" , ascending=False)

,query,query_time,rows_examined,joins
8908,SUM transactions (simulated missing index),0.279081,250188,1
8909,SUM transactions (simulated missing index),0.274041,250188,1
8907,SUM transactions (simulated missing index),0.268203,250188,1
8993,SUM transactions (simulated missing index),0.253462,250188,1
8948,SUM transactions (simulated missing index),0.243653,250188,1
...,...,...,...,...
998,SUM transactions for user_id=7331,0.019384,5,2
867,SUM transactions for user_id=6048,0.019269,13,2
684,SUM transactions for user_id=1979,0.019176,47,2
822,SUM transactions for user_id=2803,0.018859,9,2


## 10.4 Save All Metrics to CSV


In [11]:
metrics_csv_path = r"C:\Users\pc\data science\SADOP\data\slow_query_metrics_realistic.csv"
df_spike_metrics.to_csv(metrics_csv_path, index=False)
print(f"Realistic slow query metrics saved to {metrics_csv_path}")


Realistic slow query metrics saved to C:\Users\pc\data science\SADOP\data\slow_query_metrics_realistic.csv
